SETUP

In [1]:
import sys, os, math
import numpy as np
import pandas as pd
from functools import cmp_to_key

sys.path.append(os.path.abspath(os.path.join('../..')))

from utils import matrixUtils, arrayUtils, numberUtils, geometryUtils
from utils.inputDataUtils import *
from utils.stringOpUtils import *
from utils.terminalUtils import *
from utils.labelMakerUtils import *
from utils.solutionRoot import *

In [2]:
def getInputData(filename):
    data = getTuples_numbers(filename, ',')
    
    return data

EXAMPLE DATA

In [17]:
DATA_FILENAME = "example.data"

TEST DATA

In [14]:
DATA_FILENAME = "test.data"

PART 1

In [12]:
data = getInputData(DATA_FILENAME)
print(data[:10])

[[98389, 50440], [98389, 51653], [97975, 51653], [97975, 52871], [97981, 52871], [97981, 54078], [97799, 54078], [97799, 55291], [97711, 55291], [97711, 56472]]


In [14]:
def part1(data):
    mDists = []
    for a in range(len(data)-1):
        nodeA = data[a]
        for b in range(a+1, len(data)):
            nodeB = data[b]
            mDists.append((geometryUtils.getManhattanDistance(nodeA[0],nodeA[1],nodeB[0],nodeB[1]),a,b))

    mDists.sort(reverse=True)

    # print(mDists)
    largestRectangle = mDists[0]
    startNode = data[largestRectangle[1]]
    stopNode = data[largestRectangle[2]]

    # print(startNode)
    # print(stopNode)

    return (abs(startNode[0]-stopNode[0])+1)*(abs(startNode[1]-stopNode[1])+1)

result = part1(data)
print(yellow(result))

4765757080


PART 2

In [5]:
data

[[11, 1], [11, 7], [9, 7], [9, 5], [2, 5], [2, 3], [7, 3], [7, 1]]

In [8]:
def getMarker(p,c,n): 
    if p[0]<c[0] and c[1]>n[1]: #right,up
        return [c[0]-1,c[1]-1]
    elif p[0]<c[0] and c[1]<n[1]: #right,down
        return [c[0]+1,c[1]-1]
    elif p[1]<c[1] and c[0]<n[0]: #down,right
        return [c[0]+1,c[1]-1]
    elif p[1]<c[1] and c[0]>n[0]: #down,left
        return [c[0]+1,c[1]+1]
    elif p[0]>c[0] and c[1]<n[1]: #left,down
        return [c[0]+1,c[1]+1]
    elif p[0]>c[0] and c[1]>n[1]: #left,up
        return [c[0]-1,c[1]+1]
    elif p[1]>c[1] and c[0]>n[0]: #up,left
        return [c[0]-1,c[1]+1]
    # elif p[1]>c[1] and c[0]<n[0]: #up,right
    else:
        return [c[0]-1,c[1]-1]
    

In [15]:
data = getInputData(DATA_FILENAME)

df = pd.DataFrame(data)

smallestX = min(df[1])
smallestXPos = int(df.index[df[1]==smallestX][0])

data = data[smallestXPos:] + data[0:smallestXPos]

markers = [getMarker(data[-1],data[0],data[1])]

for i in range(1,len(data)-1):
    p = data[i-1]
    c = data[i]
    n = data[i+1]

    markers.append(getMarker(p,c,n))

markers.append(getMarker(data[-2],data[-1],data[0]))

print(len(markers))


496


In [30]:
def surface(a,b):
    return (abs(a[0]-b[0])+1)*(abs(a[1]-b[1])+1)

def segmentsIntersect(s1a, s1b, s2a, s2b):
    if s1a[1] == s1b[1]: # s1 is horizontal
        if s2a[1] == s2b[1]: # s2 is horizontal
            return s1a[1] == s2a[1] and ((min(s1a[0], s1b[0]) < min(s2a[0], s2b[0]) < max(s1a[0], s1b[0])) or (min(s2a[0], s2b[0]) < min(s1a[0], s1b[0]) < max(s2a[0], s2b[0])))
        else: # s2 is vertical
            return (min(s1a[0], s1b[0]) < s2a[0] < max(s1a[0], s1b[0])) and (min(s2a[1], s2b[1]) < s1a[1] < max(s2a[1], s2b[1]))
    else: # s1 is vertical
        if s2a[0] == s2b[0]: # s2 is vertical
            return s1a[0] == s2a[0] and ((min(s1a[1], s1b[1]) < min(s2a[1], s2b[1]) < max(s1a[1], s1b[1])) or (min(s2a[1], s2b[1]) < min(s1a[1], s1b[1]) < max(s2a[1], s2b[1])))
        else: # s2 is horizontal
            return (min(s2a[0], s2b[0]) < s1a[0] < max(s2a[0], s2b[0])) and (min(s1a[1], s1b[1]) < s2a[1] < max(s1a[1], s1b[1]))

    

    
def isRectangleValid(markers, a, b):

    # a---d
    # |   |
    # c---b
    
    c = (a[0],b[1])
    d = (b[0],a[1])

    for mi in range(len(markers)-1):
        if segmentsIntersect(markers[mi], markers[mi+1], a, d) or segmentsIntersect(markers[mi], markers[mi+1], d, b) or segmentsIntersect(markers[mi], markers[mi+1], b, c) or segmentsIntersect(markers[mi], markers[mi+1], c, a):
            return False

    if segmentsIntersect(markers[0], markers[-1], a, d) or segmentsIntersect(markers[0], markers[-1], d, b) or segmentsIntersect(markers[0], markers[-1], b, c) or segmentsIntersect(markers[0], markers[-1], c, a):
            return False
    
    return True
    

In [32]:
surfaces = []
for ai in range(len(data)-1):
    for bi in range(ai+1, len(data)):
        a = data[ai]
        b = data[bi]

        surfaces.append((surface(a,b),a,b))

surfaces.sort(reverse=True)

result = 0
for si in range(len(surfaces)):
    s,a,b = surfaces[si]
           
    if isRectangleValid(markers,a,b):
        result = s
        break

print(yellow(result))



1498673376


In [ ]:
def part2(data):
    result = 0

    return result

print(part2(data))

PROFILING

In [ ]:
startTS = time.time()

data = getInputData(DATA_FILENAME)
dataTS = time.time()

part1Result = part1(data)
part1TS = time.time()

part2Result = part2(data)
part2TS = time.time()

dataElapsedMs = (dataTS-startTS)*1000
part1ElapsedMs = (part1TS-dataTS)*1000
part2ElapsedMs = (part2TS-part1TS)*1000

print(f"Data loaded in %.2fms" % dataElapsedMs)
print(f"Part 1 in %.2fms" % part1ElapsedMs)
print(f"Part 2 in %.2fms" % part2ElapsedMs)
print("Total: %.2fms" % (dataElapsedMs+part1ElapsedMs+part2ElapsedMs))